# Cleaning Bulletines

I used this notebook to clean historical data about detention centers in Italy. Over the years, the Ministry of Justice used different ways to refer to the various institutes. For this reason, I used this notebook to standardize the institutes' name and do some basic cleaning for further processing.

In [1]:
import pandas as pd
import re

## Basic cleaning

In [2]:
df = pd.read_csv('../outputs/raw/bollettini_mensili_raw.csv',  dtype=str)

In [3]:
# Remove rows with no data (former headers)
df = df.dropna(subset=['detenuti_totale'])
# Removing 'Total' rows
df = df.loc[~df['regione_di_detenzione'].str.contains('Totale')]

In [4]:
# Fixing dates
# Removing 2016 (incomplete data)
df['anno'] = df['dati_aggiornati_al'].str[-4:]
df = df[df['anno'] != '2016']
df.drop(columns=['anno'], inplace=True)


months_ita_num = {
    "gennaio": "01",
    "febbraio": "02",
    "marzo": "03",
    "aprile": "04",
    "maggio": "05",
    "giugno": "06",
    "luglio": "07",
    "agosto": "08",
    "settembre": "09",
    "ottobre": "10",
    "novembre": "11",
    "dicembre": "12"
}

# # Fix Dates

def fix_dates(date_string):
    day, month_name, year = date_string.split()
    month_number = months_ita_num[month_name.lower()]
    formatted_date = f"{year}-{month_number}-{day.zfill(2)}"
    return pd.to_datetime(formatted_date)

df['dati_aggiornati_al'] = df['dati_aggiornati_al'].apply(fix_dates)

Apparently there's some duplicated values for december 2021. It seems that two different links redirect to the same exact table:

* https://www.giustizia.it/giustizia/it/mg_1_14_1.page?contentId=SST360932
* https://www.giustizia.it/giustizia/it/mg_1_14_1.page?contentId=SST365607

In [5]:
# Removing duplicate table
df = df.loc[~df['url'].str.contains('SST360932')]

In [6]:
# Cleaning numbers
columns_to_clean = [
    'capienza_regolamentare',
    'detenuti_totale',
    'detenuti_donne',
    'detenuti_stranieri'
    ]
    
for column in columns_to_clean:
    df[column] = df[column].astype(str)
    df[column] = df[column].str.extract(r'(\d+\.?\d*)')
    df[column] = df[column].fillna('0')
    df[column] = df[column].astype(int)


In [7]:
# Cleaning strings
df['sigla_provincia'] = df['sigla_provincia'].str.strip()
df['regione_di_detenzione'] = df['regione_di_detenzione'].str.capitalize().str.strip()
df['nome_istituto'] = df['nome_istituto'].str.strip().str.title().str.strip('-')
df['tipo_istituto'] = df['tipo_istituto'].str.strip()
df['descrizione'] = df['descrizione'].str.strip().str.capitalize()

It seems that all Napoli provinces (acronym: NA) was interpreted by pandas into empty values. Let's fix this.

In [8]:
df['sigla_provincia'] = df['sigla_provincia'].fillna('NA')

In [9]:
df.sample(2)

,url,dati_aggiornati_al,descrizione,regione_di_detenzione,sigla_provincia,nome_istituto,tipo_istituto,capienza_regolamentare,detenuti_totale,detenuti_donne,detenuti_stranieri
14337,https://www.giustizia.it/giustizia/it/mg_1_14_...,2018-01-31,Detenuti italiani e stranieri presenti e capie...,lazio,RM,"Roma ""Regina Coeli""",CC,620,970,0,502
6417,https://www.giustizia.it/giustizia/it/mg_1_14_...,2022-06-30,Detenuti italiani e stranieri presenti e capie...,Sicilia,PA,"Palermo ""A. Lorusso"" Pagliarelli",CC,1182,1247,61,157


In [10]:
df['tipo_istituto'].value_counts()

CC       11653
CR        4030
CCF        172
CRF        172
CL          86
ICAM        86
IP          85
EX OP        3
Name: tipo_istituto, dtype: int64

In [11]:
tipi_istituto = {
    'CC' : 'Casa circondariale',
    'CR' : 'Casa di reclusione',
    'CCF': 'Casa circondariale femminile',
    'CRF': 'Casa di reclusione femminile',
    'CL' : 'Casa di lavoro',
    'ICAM': 'Istituto custodia attenuata per madri',
    'IP' : 'Istituto di pena'
}

In [12]:
df['tipo_istituto_desc'] = df['tipo_istituto'].map(tipi_istituto)

## Standardizing names

In [13]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mdallastella/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
# Preparing names to clean
names_toclean_df = df[['nome_istituto', 'tipo_istituto_desc']]
names_toclean_df = names_toclean_df.drop_duplicates()
names_toclean_df['nome_tipo_istituto'] = names_toclean_df['nome_istituto'] + ' ' + names_toclean_df['tipo_istituto_desc']
names_toclean_df.sample(3)

,nome_istituto,tipo_istituto_desc,nome_tipo_istituto
84,Mantova,Casa circondariale,Mantova Casa circondariale
175,"Perugia ""Nuovo Complesso Penitenziario Capanne""",Casa circondariale,"Perugia ""Nuovo Complesso Penitenziario Capanne..."
148,"Palermo ""C. Di Bona"" Ucciardone",Casa di reclusione,"Palermo ""C. Di Bona"" Ucciardone Casa di reclus..."


In [15]:
# Preparing clean names
istituti_df = pd.read_csv('../outputs/clean/istituti_penitenziari.csv', encoding="utf-8-sig")
names_clean_df = istituti_df[['nome_istituto', 'tipo_istituto']]
names_clean_df = names_clean_df.drop_duplicates()
names_clean_df['nome_tipo_istituto'] = names_clean_df['nome_istituto'] + ' ' + names_clean_df['tipo_istituto']
names_clean_df.head()

,nome_istituto,tipo_istituto,nome_tipo_istituto
0,Reggio Calabria Arghillà,Casa circondariale,Reggio Calabria Arghillà Casa circondariale
1,Brescia Verziano,Casa di reclusione,Brescia Verziano Casa di reclusione
2,Bergamo,Casa circondariale,Bergamo Casa circondariale
3,Busto Arsizio,Casa circondariale,Busto Arsizio Casa circondariale
4,Como,Casa circondariale,Como Casa circondariale


In [16]:
# Creating two lists of names for tokenization
names_toclean = names_toclean_df['nome_tipo_istituto'].str.lower().unique().astype(str)
names_clean = names_clean_df['nome_tipo_istituto'].str.lower().unique().astype(str)

In [17]:
def count_tokens(name_toclean, name_clean):
    tokens1 = nltk.word_tokenize(name_clean)
    tokens2 = nltk.word_tokenize(name_toclean)
    common_tokens = set(tokens1) & set(tokens2)
    return len(common_tokens)

matches = []

for name_toclean in names_toclean:
    max_count = 0
    best_match = None
    for name_clean in names_clean:
        count = count_tokens(name_toclean, name_clean)
        if count > max_count:
            max_count = count
            best_match = name_clean
    if best_match is not None:
        matches.append({'name_toclean': name_toclean, 'name_match': best_match, 'count': max_count})
    else:
        matches.append({'name_toclean': name_toclean, 'name_match': best_match,  'count': 0})

matches_df = pd.DataFrame(matches, columns=['name_toclean', 'name_match', 'count'])

In [18]:
matches_df.to_csv('../outputs/raw/matches_temp.csv', index=False, encoding="utf-8-sig")

NLKT seems to have done a great job in matching our names. However, after a local review of the data though there's something to fix.

In [19]:
wrong_matches = [
    'alghero "giuseppe tomasiello" casa di reclusione',
    'camerino  casa circondariale'
    'camerino casa circondariale',
    'favignana "giuseppe barraco" casa di reclusione',
    "forli'  casa circondariale",
    "forli' casa circondariale",
    'lanusei "san daniele" casa circondariale',
    'lode\' "mamonelode\'\" casa di reclusione',
    'onani "mamone" casa di reclusione',
    'perugia "nuovo complesso penitenziario capanne" casa circondariale',
    'roma "r. cinotti" rebibbia n.c.1 casa circondariale',
    'roma "raffaele cinotti" rebibbia n.c.1 casa circondariale',
    'san remo "n.c." casa di reclusione',
    'sanremo  casa circondariale',
    'sanremo casa circondariale',
    'piacenza "san lazzaro" casa circondariale',
    ]


matches_df[matches_df['name_toclean'].isin(wrong_matches)].sort_values('name_match', ascending=True)

,name_toclean,name_match,count
71,"san remo ""n.c."" casa di reclusione",alessandria san michele casa di reclusione,4
129,"onani ""mamone"" casa di reclusione",brescia verziano casa di reclusione,3
350,"lode' ""mamonelode'"" casa di reclusione",brescia verziano casa di reclusione,3
335,"alghero ""giuseppe tomasiello"" casa di reclusione",civitavecchia giuseppe passerini casa di reclu...,4
344,"favignana ""giuseppe barraco"" casa di reclusione",civitavecchia giuseppe passerini casa di reclu...,4
40,forli' casa circondariale,reggio calabria arghillà casa circondariale,2
208,forli' casa circondariale,reggio calabria arghillà casa circondariale,2
304,sanremo casa circondariale,reggio calabria arghillà casa circondariale,2
306,sanremo casa circondariale,reggio calabria arghillà casa circondariale,2
61,"roma ""r. cinotti"" rebibbia n.c.1 casa circonda...",roma rebibbia iiiª casa casa circondariale,4


In [20]:
# Fixing names manually
matches_df.loc[matches_df['name_toclean'].str.contains('alghero "giuseppe tomasiello"'), 'name_match'] = "alghero casa di reclusione"
matches_df.loc[matches_df['name_toclean'].str.contains('camerino'), 'name_match'] = "camerino casa circondariale"
matches_df.loc[matches_df['name_toclean'].str.contains('favignana'), 'name_match'] = "favignana casa di reclusione"
matches_df.loc[matches_df['name_toclean'].str.contains("forli'"), 'name_match'] = "forlì casa circondariale"
matches_df.loc[matches_df['name_toclean'].str.contains('lanusei'), 'name_match'] = "lanusei casa circondariale"
matches_df.loc[matches_df['name_toclean'].str.contains("lode'"), 'name_match'] = "onanì casa di reclusione"
matches_df.loc[matches_df['name_toclean'].str.contains("onani"), 'name_match'] = "onanì casa di reclusione"
matches_df.loc[matches_df['name_toclean'].str.contains("perugia"), 'name_match'] = "perugia casa circondariale"
matches_df.loc[matches_df['name_toclean'].str.contains("capanne"), 'name_match'] = "perugia casa circondariale"
matches_df.loc[matches_df['name_toclean'].str.contains('cinotti'), 'name_match'] = "roma rebibbia nuovo complesso casa circondariale"
matches_df.loc[matches_df['name_toclean'].str.contains('san remo'), 'name_match'] = "sanremo casa di reclusione"
matches_df.loc[matches_df['name_toclean'].str.contains('sanremo'), 'name_match'] = "sanremo casa di reclusione"
matches_df.loc[matches_df['name_toclean'].str.contains('piacenza'), 'name_match'] = "piacenza casa circondariale"
matches_df.loc[matches_df['name_toclean'].str.contains('reggio emilia'), 'name_match'] = "reggio emilia istituto di pena"




In [21]:
matches_df[matches_df['name_toclean'].isin(wrong_matches)].sort_values('name_match', ascending=True)

,name_toclean,name_match,count
335,"alghero ""giuseppe tomasiello"" casa di reclusione",alghero casa di reclusione,4
344,"favignana ""giuseppe barraco"" casa di reclusione",favignana casa di reclusione,4
40,forli' casa circondariale,forlì casa circondariale,2
208,forli' casa circondariale,forlì casa circondariale,2
127,"lanusei ""san daniele"" casa circondariale",lanusei casa circondariale,3
129,"onani ""mamone"" casa di reclusione",onanì casa di reclusione,3
350,"lode' ""mamonelode'"" casa di reclusione",onanì casa di reclusione,3
175,"perugia ""nuovo complesso penitenziario capanne...",perugia casa circondariale,4
43,"piacenza ""san lazzaro"" casa circondariale",piacenza casa circondariale,3
61,"roma ""r. cinotti"" rebibbia n.c.1 casa circonda...",roma rebibbia nuovo complesso casa circondariale,4


In [22]:
df['nome_tipo_istituto'] = df['nome_istituto'].str.lower() + " " + df['tipo_istituto_desc'].str.lower()
df['nome_tipo_istituto'] = df['nome_tipo_istituto'].str.strip().astype(str)

matches_df['name_match'] = matches_df['name_match'].str.strip().astype(str)

# Merge df with matches_df on nome_tipo_istituto and name_match
merged_df = df.merge(matches_df, left_on='nome_tipo_istituto', right_on='name_toclean', how='left')



In [23]:
# Drop unneccessary columns
merged_df = merged_df.drop('count', axis=1)
merged_df = merged_df.drop('name_toclean', axis=1)
merged_df = merged_df.drop('nome_tipo_istituto', axis=1)

merged_df.head(2)

,url,dati_aggiornati_al,descrizione,regione_di_detenzione,sigla_provincia,nome_istituto,tipo_istituto,capienza_regolamentare,detenuti_totale,detenuti_donne,detenuti_stranieri,tipo_istituto_desc,name_match
0,https://www.giustizia.it/giustizia/it/mg_1_14_...,2021-01-31,Detenuti italiani e stranieri presenti e capie...,Abruzzo,AQ,Avezzano,CC,53,61,0,14,Casa circondariale,avezzano casa circondariale
1,https://www.giustizia.it/giustizia/it/mg_1_14_...,2021-01-31,Detenuti italiani e stranieri presenti e capie...,Abruzzo,AQ,L'Aquila,CC,235,188,13,18,Casa circondariale,l'aquila casa circondariale


In [24]:
merged_df['nome_istituto'] = merged_df['name_match']
merged_df['nome_istituto'] = merged_df['nome_istituto'].str.title()

# Drop unneccessary columns
merged_df = merged_df.drop('tipo_istituto_desc', axis=1)
merged_df = merged_df.drop('name_match', axis=1)

In [25]:
merged_df.sample(5)

,url,dati_aggiornati_al,descrizione,regione_di_detenzione,sigla_provincia,nome_istituto,tipo_istituto,capienza_regolamentare,detenuti_totale,detenuti_donne,detenuti_stranieri
6950,https://www.giustizia.it/giustizia/it/mg_1_14_...,2022-03-31,Detenuti italiani e stranieri presenti e capie...,Sicilia,ME,Messina Casa Circondariale,CC,300,209,37,23
4868,https://www.giustizia.it/giustizia/it/mg_1_14_...,2023-02-28,Detenuti italiani e stranieri presenti e capie...,Sicilia,EN,Enna Casa Circondariale,CC,169,201,0,35
5370,https://www.giustizia.it/giustizia/it/mg_1_14_...,2022-11-30,Detenuti italiani e stranieri presenti e capie...,Lombardia,LC,Lecco Casa Circondariale,CC,53,72,0,38
15773,https://www.giustizia.it/giustizia/it/mg_1_14_...,2017-03-31,Detenuti italiani e stranieri presenti e capie...,lazio,RM,Civitavecchia Nuovo Complesso Casa Circondariale,CC,344,446,30,261
10181,https://www.giustizia.it/giustizia/it/mg_1_14_...,2019-09-30,Detenuti italiani e stranieri presenti e capie...,toscana,LI,Porto Azzurro Casa Di Reclusione,CR,337,388,0,242


In [26]:
merged_df = merged_df.drop_duplicates()
merged_df.to_csv('../outputs/clean/bollettini_mensili_data.csv', index=False, encoding="UTF-8-sig")